In [127]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

SUPABASE_KEY = os.getenv("SUPABASE_KEY")

url = f"https://iukxcgvmzjfelwfrpkyi.supabase.co/rest/v1/analytics?apikey={SUPABASE_KEY}&select=*"
headers = {
    "apikey": SUPABASE_KEY,
    "Authorization": f"Bearer {SUPABASE_KEY}",
}

response = requests.get(url, headers=headers)

# Create a dataframe from the JSON response payload
data = response.json()


In [128]:
df = pd.DataFrame(data)
blacklisted_user_ids = ["390c0190-33a3-4029-ae09-687bff0b77d6", "42a0286f-f933-490a-89aa-0c0037dae11e"]
should_delete = df[df["type"] == "delete-data"]
blacklisted_user_ids.extend(should_delete["user_id"].tolist())
# Exclude all events where the url contains "music.apple"
df = df[~df["url"].str.contains("music\.apple", na=False)]

# Filter out rows where user_id is in the blacklisted_user_ids array
df = df[~df["user_id"].isin(blacklisted_user_ids)]


def get_top_domain(domain: str) -> str:
    if domain.startswith("www") or domain.startswith("ww2"):
        domain = domain[4:]
    
    d = domain.split("/")[0]

    return ".".join(d.split(".")[-3:]) if ".co.uk" in d else ".".join(d.split(".")[-2:])

# Add column that says if the user has a strategy or not
df["domain"] = df["url"].apply(lambda x: get_top_domain(x) if x != None else None)
# Define an empty array for blacklisted user ids


# AFTER:  2025-04-13 02:00
# BEFORE: 2025-04-26 10:00
df = df[(df["created_at"] >= "2025-03-27 01:00")]

In [ ]:
shop_df = df[df["domain"] == "shop.app"]

# Count shop_df that contains /checkout and how many doesnt
checkout_df = shop_df[shop_df["url"].str.contains("/checkout|/pay/")]
no_checkout_df = shop_df[~shop_df["url"].str.contains("/checkout|/pay/")]

before_checkout_df_len = len(checkout_df)
before_no_checkout_df_len = len(no_checkout_df)

print("Filter shop.app URLs")

user_df = df.groupby("user_id")

# for each user_id iterate through the rows using index
for user_id, rows in user_df:
    # for each row in the user_id group, if url contains /checkout and is domain shop.app,
    # then set domain to the previous domain
    for index, row in rows.iterrows():
        if row["domain"] == "shop.app" and ("/checkout" in row["url"] or "/pay/" in row["url"]):
            # Ensure index - 1 is valid
            if previous_domain is not None:
                df.at[row.name, "domain"] = previous_domain
        else:
            previous_domain = row["domain"]

shop_df = df[df["domain"] == "shop.app"]

# Count shop_df that contains /checkout and how many doesnt
checkout_df = shop_df[shop_df["url"].str.contains("/checkout|/pay/")]
no_checkout_df = shop_df[~shop_df["url"].str.contains("/checkout|/pay/")]

 

# print
print(f"Number of shop.app URLs with /checkout filtered from {before_checkout_df_len} to {len(checkout_df)}")
print(f"Number of shop.app URLs without /checkout filtered from {before_checkout_df_len} to {len(checkout_df)}")


Filter shop.app URLs
Number of shop.app URLs with /checkout filtered from 0 to 183
Number of shop.app URLs without /checkout filtered from 9 to 183


In [79]:
# TO CSV
df.to_csv("analytics.csv", index=False)


In [80]:
# List all with type = "place-order"
df[df["type"] == "place-order"]


,id,type,url,payload,user_id,session_id,received_at,created_at,domain


In [81]:
# Create a new data frame, where it only includes a single sample of each type of event. The event is the "type" column.
df_unique = df.drop_duplicates(subset=["type"])
df_unique


,id,type,url,payload,user_id,session_id,received_at,created_at,domain
1552,19636,on-onboarding,www.lessextension.com/onboarding,"""""",none,none,2025-03-27T15:23:55.135162+00:00,2025-03-27T15:23:51.868+00:00,lessextension.com
1553,19648,time-spent,www.ticketmaster.dk/checkout/Z698xZC4Z1744-y/5...,"{""duration"":4717}",0b7cd927-52ef-434b-8b2e-9c63281c6327,8edc93bd-6b53-402b-be22-a038953822c8,2025-03-27T15:28:02.023488+00:00,2025-03-27T15:28:01.461+00:00,ticketmaster.dk
1558,19704,from-directs,www.lessextension.com/,"""kua""",less-website,none,2025-03-28T07:33:48.938317+00:00,2025-03-28T07:33:48.628+00:00,lessextension.com
1575,19637,page-view,www.ticketmaster.dk/event/555879,None,0b7cd927-52ef-434b-8b2e-9c63281c6327,e32b5434-2806-4a2e-a90e-ba0ede64bf83,2025-03-27T15:26:56.84443+00:00,2025-03-27T15:26:55.654+00:00,ticketmaster.dk
1580,19642,welcome-modal-seen,www.ticketmaster.dk/event/555879,None,0b7cd927-52ef-434b-8b2e-9c63281c6327,e32b5434-2806-4a2e-a90e-ba0ede64bf83,2025-03-27T15:27:14.463082+00:00,2025-03-27T15:27:13.93+00:00,ticketmaster.dk
2516,19701,from-directs-cta,www.lessextension.com/,"""shoppingaddiction""",less-website,none,2025-03-27T21:32:22.421951+00:00,2025-03-27T21:32:22.176+00:00,lessextension.com
2770,19722,uninstall,www.lessextension.com/goodbye,None,1e42348b-77f0-4eb2-911b-93e0e27bd8a0,none,2025-03-29T16:14:40.952129+00:00,2025-03-29T16:14:36.678+00:00,lessextension.com
2827,19779,open-popup,kcgblchgejkpnemehaojecgbamdiacml/popup.html,None,5c7c5d5f-ad28-401e-b6f0-6d6f167cc726,0218f027-c5c1-410a-8ac9-aae33683e886,2025-03-31T09:32:24.938499+00:00,2025-03-31T09:32:24.233+00:00,kcgblchgejkpnemehaojecgbamdiacml
3056,19971,add-to-cart,www.amazon.com/gp/product/1108724264/ref=as_li_tl,None,a584c10c-d236-408a-b544-50b6219dee39,b49b0119-a045-45fe-8476-80d7e2d91bad,2025-04-02T12:11:11.798597+00:00,2025-04-02T12:11:11.089+00:00,amazon.com
3759,20650,active,gckehbmaalopcmjjbimcphlncjbnaakd/popup.html,false,10fb5e0c-f579-4234-ab46-b609a9cfb5d0,109592d0-8706-4744-8269-f2960df6281e,2025-04-03T07:42:38.57899+00:00,2025-04-03T07:42:38.37+00:00,gckehbmaalopcmjjbimcphlncjbnaakd


In [82]:
# count different types of events
event_counts = df["type"].value_counts()
print(event_counts)

type
time-spent                    7495
page-view                      745
from-directs                   213
on-onboarding                  171
from-directs-cta                42
add-to-cart                     36
uninstall                       27
open-popup                      25
welcome-modal-seen              17
active                          15
enforce_wait_modal_shown        14
enforce_wait_info_expanded       7
enforce_wait_canceled            6
questionary-popup                4
open-options                     3
Name: count, dtype: int64


In [83]:
import re

# Count unique user IDs
# Define a regex pattern for UUIDv4
uuidv4_pattern = re.compile(r'^[a-f0-9]{8}-[a-f0-9]{4}-4[a-f0-9]{3}-[89ab][a-f0-9]{3}-[a-f0-9]{12}$', re.IGNORECASE)

# Filter user_id's that match the UUIDv4 pattern
uuidv4 = df[df["user_id"].apply(lambda x: bool(uuidv4_pattern.match(x)))]
unique_users = uuidv4["user_id"].nunique()

uninstalled = df[df["type"] == "uninstall"]
unique_uninstalled = uninstalled["user_id"].nunique()
unique_installed = unique_users - unique_uninstalled

# look at the last active event for each unique user. If the last event has a payload of "false", this means they've deactivated it.
# I'd like to know how many of the current installed users have deactivated the extension.
last_active = df[df["type"] == "active"]
last_active = last_active.drop_duplicates(subset=["user_id"], keep="last")
last_active = last_active[last_active["payload"] == "false"]
unique_deactivated = last_active["user_id"].nunique()
totally_active = unique_installed - unique_deactivated
on_onboarding = df[(df["type"] == "on-onboarding") & (df["payload"].apply(lambda x: x != '""'))]

print("Installations:", len(on_onboarding), "\nUser activity seen from", unique_users, "\nuninstalled:", unique_uninstalled, "\ndeactivated:", unique_deactivated, "\nTotal registered active users:", totally_active)

Installations: 8 
User activity seen from 53 
uninstalled: 27 
deactivated: 3 
Total registered active users: 23
